In [1]:
import re
from pathlib import Path
import pandas as pd

data_folder = Path("C:/Users/Jing Li/Desktop/cv2db/project/Inputs")
data_to_open = data_folder / "AllResume.txt"

In [2]:
with open(data_to_open) as f: 
	data = f.read()
    
# Seperate Qualification and Experience from the rest
pattern = r"Angela Bowden – Training Specialist"
temp = list(filter(None, re.split(pattern, data)))
QE = temp[0]
data = pattern + temp[1]
data = re.sub(r"Qualifications and Experience \n  Over 30 years of Data Architecture and \
                information systems design \n BS Chemical Engineering, Imperial \nCollege,\
                London", '\n', data)

In [3]:
#Checking
# QE
# data

In [4]:
pattern_QE = r"\nQualifications and Experience"

QEs = re.split(pattern_QE, QE)
QEs = list(filter(None, QEs))
# print(len(QEs))

In [5]:
# Split: name and title

pattern_ppl = r"[^\n]+\nCareer Summary"
pattern_name = r"\s–\s"

ppls_noname = re.split(pattern_ppl, data)
ppls_noname = list(filter(None, ppls_noname))

name_title = re.findall(pattern_ppl, data)

name = [re.split(pattern_name, i)[0] for i in name_title]
for i in [4, 17, 18]:
    name[i] = name[i].split(",")[0]

title = [re.split(pattern_name, i)[1] for i in name_title]
title1 = [re.sub(r' \nCareer Summary', '', i) for i in title] #remove ' \nCareer Summary' from title

In [6]:
#Checking

# print(len(name))
# print(len(title))
# print(len(title1))

In [7]:
# Split: career summary, ETC (Education, Training, and Certifications)

#Career Summary
pattern_crr_smmry = r"\nEducation, Training, and Certifications"
crr_smmry = [re.split(pattern_crr_smmry, i)[0] for i in ppls_noname]
crr_smmry = [i.replace("\n", " ") for i in crr_smmry] #career summary
ppls1 = [re.split(pattern_crr_smmry, i)[1] for i in ppls_noname] #ETC & Work Experience

# print(len(ppls1)) #check

#ETC
pattern_ETC = r"Work Experience *\n"
ETC = [re.split(pattern_ETC, i)[0] for i in ppls1] #Education, Training, and Certifications
ppls2 = [re.split(pattern_ETC, i)[1] for i in ppls1]

In [8]:
#Split Work Experience

#Work Experience
pattern_Experience = r"(\n(Publications and Awards|Awards\/Honors|Publications|Research and Publications))"
ppls2_list = [re.split(pattern_Experience, i) for i in ppls2]
for i in ppls2_list:
    i.extend([None,None,None])
wrk_exp = [i[0] for i in ppls2_list] #Work Experience
pub = [i[3] for i in ppls2_list] #Hornors/Awards/Publications


# further seperate work experience
pattern_exp = r"\n[^\n]+\d{4}\s?[–-]"
wrk_exp_list = [re.split(pattern_exp, i) for i in wrk_exp] # a list of lists of work description
wrk_exp_list1 = [re.findall(pattern_exp, i) for i in wrk_exp] # a list of lists of work_title/time

#Check
# print ([len(i) for i in wrk_exp_list])
# print ([len(i) for i in wrk_exp_list1])

for i in range(len(wrk_exp_list)):
    # wrk_exp_list[i] is a list containing work experiences of i-th consultant
    for j in range(len(wrk_exp_list1[i])):
        wrk_exp_list[i][j+1] = wrk_exp_list1[i][j]+ wrk_exp_list[i][j+1]
        
#Check
# print(wrk_exp_list)
# print ([len(i) for i in wrk_exp_list])

In [9]:
# Build a simple dataframe to store the chunked information

df1 = pd.DataFrame({"Source": "All",
                    "Name":name,
                    "Title": title1,
                    "Qualification and Experience": QEs,
                    "Career Summary": crr_smmry,
                    "ETC": ETC,
                    "Work Experience": wrk_exp_list,
                    "Hornors/Awards/Publications": pub,
                    'ppls_noname': ppls_noname}) # for extract keywords, but repetive information (no name, title, QE)

In [80]:
df1

,Source,Name,Title,Qualification and Experience,Career Summary,ETC,Work Experience,Hornors/Awards/Publications,ppls_noname
0,All,Angela Bowden,Training Specialist,\n•\tOver 20 years in Training \nManagement a...,Driven learning professional with 16 years ...,"\n• University of Florida, Master of Science ...",[Program Manager \t \t \t \t \t \t \t \tMay 20...,\n• HRSA Director’s Award for Technical Assis...,\nDriven learning professional with 16 years...
1,All,Carly Mensah,Project Manager,\n•\tOver 10 years of experience in project m...,An accomplished professional with over 10 y...,"\n• University of Maryland, Bachelor of Scien...",[Project Manager \t \t \t \t \t \t \t ...,None,\nAn accomplished professional with over 10 ...
2,All,Claire Pak,Senior Writer Editor,\n•\tOver two years of experience supporting ...,Claire Pak is a writer and editor with seve...,"\nUniversity of Maryland University College, ...",[Communications Manager/Technical Writer \t \...,"\n• DSFederal Employee of the Year, 2017 \n• ...",\nClaire Pak is a writer and editor with sev...
3,All,Celeste Pleasant,Research Analyst/Advisor,\n•\tOver 20 years of technical assistance an...,A highly organized professional with over 2...,"\n• Howard University, Bachelor of Science \n \n",[Senior Meeting Planner \t \t \t \t \t \tApril...,None,\nA highly organized professional with over ...
4,All,Echo Wang,Senior Researcher/Clinician,\n•\tMore than eight years of experience mana...,Ms. Echo Wang is an experienced professiona...,"\n• Doctor in Public Health (DrPH), Health Po...",[Project Manager/Principle Investigator \t \t ...,"\n• Wang HE, Lee M, Hart A, Summers AC...",\nMs. Echo Wang is an experienced profession...
5,All,Frank Van Balen,Corporate Monitor,\n•\tOver 20 years of corporate and project f...,Mr. Frank Van Balen is a finance specialist...,"\nB.S. Finance, Virginia Polytechnic Institut...",[Business Finance Manager \t \t \t \t \t \t0...,\n• HRSA Director’s Award for Technical Assis...,\nMr. Frank Van Balen is a finance specialis...
6,All,Jessica Chen,Health Informatics Analyst,\n• 3.5 years of business analysis experience...,Ms. Jessica Chen is an experienced analyst ...,"\n• McGill University, B.S., Chemistry, Minor...",[Business Analyst \tMarch 2014 – Present \nDSF...,"\n• Honorable Mention, Employee of the Year, ...",\nMs. Jessica Chen is an experienced analyst...
7,All,Nigel Shaw,Data Architect,\n•\tOver 30 years of Data Architeture and in...,"Systems architecture, analysis and design, ...","\nBS Chemical Engineering Part 1, Imperial Co...",[Solutions Architect \t \t \t \t \t \tJanuar...,None,"\nSystems architecture, analysis and design,..."
8,All,Yinong Chong,Data Administrator/Statistician,\n•\tOver 20 years of scientific and evaluati...,As a senior health scientist with 20 years’...,"\n• Project Management Institute, Project Man...",[Principal health data scientist \t \t \t \t ...,\nAward highlights: \n• CDC “Excellence in In...,\nAs a senior health scientist with 20 years...
9,All,Xiaozhe (Katherine) Miao,Billing Specialist,"\n•\tPassed all CPA exams, license applicable...",An experienced billing specialist working o...,"\n• University of Michigan, Master of Account...",[Billing Specialist \t \t \t \t \t \t \t ...,"\n• Honored as a member of Beta Gamma Sigma, ...",\nAn experienced billing specialist working ...


In [48]:
#Storing the dataframe

file_name = data_folder / "intermediate/All_Phase1.pkl"
df1.to_pickle(file_name)